In [ ]:
import os
import sys
sys.path.append("../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
)
from utils.model_utils.save_module import save_module
from utils.model_utils.load_model import load_model
from utils.model_utils.evaluate import evaluate_model, get_sparsity
from utils.dataset_utils.sampling import SamplingDataset
from utils.prune_utils.prune import (
    prune_concern_identification,
    recover_tangling_identification
)
from utils.prune_utils.prune_head import prune_head

In [ ]:
name = "OSDG"
device = torch.device("cuda:0")
checkpoint = None
batch_size=32
num_workers=48
num_samples=16
concern=0
ci_ratio=0.5
ti_ratio=0.1
include_layers=["attention", "intermediate", "output"]

In [ ]:
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]
model, tokenizer, checkpoint = load_model(model_config)

In [ ]:
model, tokenizer, checkpoint = load_model(model_config)

In [ ]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    name, batch_size=batch_size, num_workers=num_workers
)

In [ ]:
head_list = [(4, 9), (5, 1), (8, 0), (5, 10), (8, 9), (2, 2), (11, 5), (2, 11), (6, 2), (7, 1),
               (6, 11), (7, 10), (4, 2), (3, 6), (5, 3), (8, 2), (9, 1), (9, 10), (0, 7), (2, 4),
               (1, 8), (6, 4), (7, 3), (3, 8), (5, 5), (8, 4), (9, 3), (0, 0), (11, 9), (1, 10),
               (6, 6), (3, 1), (3, 10), (5, 7), (0, 2), (11, 2), (1, 3), (7, 7), (3, 3), (5, 0),
               (5, 9), (4, 11), (9, 7), (10, 8), (1, 5), (6, 1), (7, 9), (3, 5), (5, 2), (9, 0),
               (5, 11), (9, 9), (10, 1), (10, 10), (1, 7), (2, 6), (7, 2), (7, 11), (3, 7), (5, 4),
               (4, 6), (9, 2), (8, 6), (10, 3), (1, 0), (1, 9), (0, 11), (2, 8), (7, 4), (3, 9),
               (5, 6), (4, 8), (10, 5), (1, 2), (0, 4), (2, 1), (1, 11), (6, 10), (3, 2), (4, 1),
               (3, 11), (4, 10), (8, 1), (8, 10), (1, 4), (0, 6), (2, 3), (6, 3), (3, 4), (4, 3),
               (8, 3), (10, 0), (10, 9), (1, 6), (0, 8), (2, 5), (9, 11), (6, 5), (4, 5), (8, 5),
               (10, 2), (9, 4), (0, 1), (11, 1), (0, 10), (2, 7), (11, 10), (7, 6), (4, 7), (5, 8),
               (8, 7), (1, 1), (0, 3), (2, 0), (9, 6), (11, 3), (2, 9), (6, 0), (6, 9)]

In [ ]:
module = copy.deepcopy(model)

In [ ]:
prune_head(module, head_list)

In [ ]:
prune_concern_identification(
    module,
    model_config,
    positive_samples,
    negative_samples,
    include_layers=["intermediate", "output"],
    exclude_layers=["attention"],
    sparsity_ratio=ci_ratio,
)

In [ ]:
recover_tangling_identification(
    model,
    module,
    model_config,
    negative_samples,
    recovery_ratio=ti_ratio,
    include_layers=include_layers,
)

In [ ]:
similar(model, module, valid_dataloader, concern, num_samples, num_labels, device=device)

In [ ]:
result = evaluate_model(module, model_config, test_dataloader)
get_sparsity(module)